In [1]:
#!pip install icypher
#!pip install neo4j
#!pip install py2neo
#!pip install ipycytoscape
#!pip install neo4jupyter

In [2]:
import random
import ipycytoscape
from py2neo import Graph, Node, Relationship

In [3]:
# style for nodes with node names
node_centered = {'selector': 'node',
                 'style': {'font-size': '10px',
                           'label': 'data(name)',
                           'height': '60px',
                           'width': '80px',
                           'text-max-width': '80px',
                           'text-wrap': 'wrap',
                           'text-valign': 'center',
                           'background-color': 'blue',
                           'background-opacity': 0.6}
                }

# style for directed graph with arrows
edge_directed = {'selector': 'edge',
                 'style':  {'line-color': '#9dbaea',
                            'curve-style': 'bezier',
                            'target-arrow-shape': 'triangle',
                            'target-arrow-color': '#9dbaea'}
                }

# style for directed graph with arrows and relationship names
edge_directed_named = {'selector': 'edge',
                       'style':  {'font-size': '8px',
                                  'label': 'data(name)',
                                  'line-color': '#9dbaea',
                                  'text-rotation': 'autorotate',
                                  'curve-style': 'bezier',
                                  'target-arrow-shape': 'triangle',
                                  'target-arrow-color': '#9dbaea'}
                      }

# style for undirected graph
edge_undirected = {'selector': 'edge',
                   'style':  {'line-color': '#9dbaea'}
                  }

In [4]:
def random_color_palette(n_colors, seed=3):
    """ 
    Creates a random color palette of n_colors 
    See https://stackoverflow.com/questions/28999287/generate-random-colors-rgb)
    
    """
    random.seed(seed)
    return ['#'+''.join([random.choice('0123456789ABCDEF') for j in range(6)]) for i in range(n_colors)]

In [5]:
#change everytime for new project
graph = Graph("bolt://3.239.37.72:7687", user="neo4j", password="commendation-routes-winters")

In [6]:
graph.run("CREATE CONSTRAINT ON (c:Client) ASSERT c.id IS UNIQUE;")
graph.run("CREATE CONSTRAINT ON (b:Bank) ASSERT b.id IS UNIQUE;")
graph.run("CREATE CONSTRAINT ON (m:Merchant) ASSERT m.id IS UNIQUE;")
graph.run("CREATE CONSTRAINT ON (m:Mule) ASSERT m.id IS UNIQUE;")
graph.run("CREATE CONSTRAINT ON (e:Email) ASSERT e.email IS UNIQUE;")
graph.run("CREATE CONSTRAINT ON (s:SSN) ASSERT s.ssn IS UNIQUE;")
graph.run("CREATE CONSTRAINT ON (p:Phone) ASSERT p.phoneNumber IS UNIQUE;")

(No data)

In [7]:
#import client 
client  = """
LOAD CSV WITH HEADERS
FROM 'https://raw.githubusercontent.com/18Grace18/ITCS498-Special-Topics_in_Computer_Science/main/clients.csv'
AS row 
MERGE (c:Client {clientID: row.ID})
ON CREATE SET
c.email = row.EMAIL,
c.id= row.ID,
c.isfraud = row.ISFRAUD,
c.name = row.NAME,
c.phoneNumber = row.PHONENUMBER,
c.ssn = row.SSN
"""

graph.run(client)

(No data)

In [8]:
unwind_email = graph.begin()
unwind_email.run("""
MATCH (c:Client)
UNWIND c.email AS email
WITH  email, collect(c) AS emails
MERGE (e:Email {email:email})
WITH e, emails
UNWIND emails AS c
WITH e,c
MERGE (c)-[:HAS_EMAIL]->(e);
""")
unwind_email.run("""MATCH (c:Client) SET c.email = null""")

graph.commit(unwind_email)

In [9]:
unwind_ssn = graph.begin()
unwind_ssn.run("""
MATCH (c:Client)
UNWIND c.ssn AS ssn
WITH  ssn, collect(c) AS ssns
MERGE (s:SSN {ssn:ssn})
WITH s, ssns
UNWIND ssns AS c
WITH s,c
MERGE (c)-[:HAS_SSN]->(s);
""")
unwind_ssn.run("""MATCH (c:Client) SET c.ssn = null""")

graph.commit(unwind_ssn)

In [10]:
unwind_phone = graph.begin()
unwind_phone.run("""
MATCH (c:Client)
UNWIND c.phoneNumber AS phoneNumber
WITH  phoneNumber, collect(c) AS phoneNumbers
MERGE (p:Phone {phoneNumber:phoneNumber})
WITH p, phoneNumbers
UNWIND phoneNumbers AS c
WITH p,c
MERGE (c)-[:HAS_PHONE]->(p);
""")
unwind_phone.run("""MATCH (c:Client) SET c.phoneNumber = null""")

graph.commit(unwind_phone)

In [11]:
#import merchant
merchant = """
LOAD CSV WITH HEADERS
FROM 'https://raw.githubusercontent.com/18Grace18/ITCS498-Special-Topics_in_Computer_Science/main/merchants.csv'
AS row 
MERGE (m:Merchant {merchantID: row.ID})
ON CREATE SET
m.highrisk = row.HIGHRISK,
m.id = row.ID,
m.name = row.NAME
"""

graph.run(merchant)

(No data)

In [12]:
#import bank
bank = """
LOAD CSV WITH HEADERS
FROM 'https://raw.githubusercontent.com/18Grace18/ITCS498-Special-Topics_in_Computer_Science/main/transactions.csv'
AS row
WITH row WHERE row.TYPEDEST = "BANK"
MERGE (b:Bank {bankID: row.IDDEST})
ON CREATE SET
b.id = row.IDDEST,
b.name = row.NAMEDEST
"""

graph.run(bank)

(No data)

In [13]:
#import mule
mule1 = """
LOAD CSV WITH HEADERS
FROM 'https://raw.githubusercontent.com/18Grace18/ITCS498-Special-Topics_in_Computer_Science/main/transactions.csv'
AS row
WITH row WHERE row.TYPEDEST = "MULE"
MATCH (c:Client {id: row.IDDEST})
SET c:Mule
"""

graph.run(mule1)

(No data)

In [14]:
mule2 = """
LOAD CSV WITH HEADERS
FROM 'https://raw.githubusercontent.com/18Grace18/ITCS498-Special-Topics_in_Computer_Science/main/transactions.csv'
AS row
WITH row WHERE row.TYPEORIG = "MULE"
MATCH (c:Client {id: row.IDORIG})
SET c:Mule
"""

graph.run(mule2)

(No data)

In [15]:
#import relationships
#CASH_IN: Client -> Merchant
#CASH_OUT: Client -> Merchant
#PAYMENT: Client -> Merchant
#TRANSFER: Client -> Client or Mule 
#DEBIT: Client -> Bank

cash_in = """
LOAD CSV WITH HEADERS
FROM 'https://raw.githubusercontent.com/18Grace18/ITCS498-Special-Topics_in_Computer_Science/main/transactions.csv'
AS row
WITH row WHERE row.ACTION = "CASH_IN" AND row.TYPEORIG = "CLIENT" AND row.TYPEDEST = "MERCHANT"
MATCH (o:Client {id: row.IDORIG})
MATCH (d:Merchant {id: row.IDDEST})
MERGE (o)-[r:CASH_IN]->(d)
ON CREATE
SET r.amount=toInteger(row.AMOUNT),
r.id=row.GLOBALSTEP,
r.isFraud = row.ISFRAUD,
r.nameDest = row.NAMEDEST,
r.nameOrig = row.NAMEORIG
"""

graph.run(cash_in)

(No data)

In [16]:
cash_out = """
LOAD CSV WITH HEADERS
FROM 'https://raw.githubusercontent.com/18Grace18/ITCS498-Special-Topics_in_Computer_Science/main/transactions.csv'
AS row
WITH row WHERE row.ACTION = "CASH_OUT" AND row.TYPEORIG = "CLIENT" AND row.TYPEDEST = "MERCHANT"
MATCH (o:Client {id: row.IDORIG})
MATCH (d:Merchant {id: row.IDDEST})
MERGE (o)-[r:CASH_OUT]->(d)
ON CREATE
SET r.amount=toInteger(row.AMOUNT),
r.id=row.GLOBALSTEP,
r.isFraud = row.ISFRAUD,
r.nameDest = row.NAMEDEST,
r.nameOrig = row.NAMEORIG
"""

graph.run(cash_out)

(No data)

In [17]:
payment = """
LOAD CSV WITH HEADERS
FROM 'https://raw.githubusercontent.com/18Grace18/ITCS498-Special-Topics_in_Computer_Science/main/transactions.csv'
AS row
WITH row WHERE row.ACTION = "PAYMENT" AND row.TYPEORIG = "CLIENT" AND row.TYPEDEST = "MERCHANT"
MATCH (o:Client {id: row.IDORIG})
MATCH (d:Merchant {id: row.IDDEST})
MERGE (o)-[r:PAYMENT]->(d)
ON CREATE
SET r.amount=toInteger(row.AMOUNT),
r.id=row.GLOBALSTEP,
r.isFraud = row.ISFRAUD,
r.nameDest = row.NAMEDEST,
r.nameOrig = row.NAMEORIG
"""

graph.run(payment)

(No data)

In [18]:
debit = """
LOAD CSV WITH HEADERS
FROM 'https://raw.githubusercontent.com/18Grace18/ITCS498-Special-Topics_in_Computer_Science/main/transactions.csv'
AS row
WITH row WHERE row.ACTION = "DEBIT" AND row.TYPEORIG = "CLIENT" AND row.TYPEDEST = "BANK"
MATCH (o:Client {id: row.IDORIG})
MATCH (d:Bank {id: row.IDDEST})
MERGE (o)-[r:DEBIT]->(d)
ON CREATE
SET r.amount=toInteger(row.AMOUNT),
r.id=row.GLOBALSTEP,
r.isFraud = row.ISFRAUD,
r.nameDest = row.NAMEDEST,
r.nameOrig = row.NAMEORIG
"""

graph.run(debit)

(No data)

In [19]:
transfer_client = """
LOAD CSV WITH HEADERS
FROM 'https://raw.githubusercontent.com/18Grace18/ITCS498-Special-Topics_in_Computer_Science/main/transactions.csv'
AS row
WITH row WHERE row.ACTION = "TRANSFER" AND row.TYPEORIG = "CLIENT" AND row.TYPEDEST = "CLIENT"
MATCH (o:Client {id: row.IDORIG})
MATCH (d:Client {id: row.IDDEST})
MERGE (o)-[r:TRANSFER]->(d)
ON CREATE
SET r.amount=toInteger(row.AMOUNT),
r.id=row.GLOBALSTEP,
r.isFraud = row.ISFRAUD,
r.nameDest = row.NAMEDEST,
r.nameOrig = row.NAMEORIG
"""

graph.run(transfer_client)

(No data)

In [20]:
transfer_mule = """
LOAD CSV WITH HEADERS
FROM 'https://raw.githubusercontent.com/18Grace18/ITCS498-Special-Topics_in_Computer_Science/main/transactions.csv'
AS row
WITH row WHERE row.ACTION = "TRANSFER" AND row.TYPEORIG = "CLIENT" AND row.TYPEDEST = "MULE"
MATCH (o:Client {id: row.IDORIG})
MATCH (d:Mule {id: row.IDDEST})
MERGE (o)-[r:TRANSFER]->(d)
ON CREATE
SET r.amount=toInteger(row.AMOUNT),
r.id=row.GLOBALSTEP,
r.isFraud = row.ISFRAUD,
r.nameDest = row.NAMEDEST,
r.nameOrig = row.NAMEORIG
"""
graph.run(transfer_mule)

(No data)

In [21]:
#Identify clients sharing PII
shared_pii = """
MATCH (c1:Client)-[:HAS_EMAIL|:HAS_PHONE|:HAS_SSN]->(n) <-[:HAS_EMAIL|:HAS_PHONE|:HAS_SSN]-(c2:Client)
WHERE id(c1) < id(c2)
RETURN c1.id, c2.id, count(*) AS freq
ORDER BY freq DESC;
"""

graph.run(shared_pii).to_data_frame()

,c1.id,c2.id,freq
0,4627465846191385,4040883736607030,2
1,4511492141916398,4101656599767539,1
2,4034960908626119,4762898444698945,1
3,4970667320743580,4997299681357575,1


In [22]:
unique_pii = """
MATCH (c1:Client)-[:HAS_EMAIL|:HAS_PHONE|:HAS_SSN]->(n) <-[:HAS_EMAIL|:HAS_PHONE|:HAS_SSN]-(c2:Client)
WHERE id(c1) <> id(c2)
RETURN count(DISTINCT c1.id) AS freq;"""

graph.run(unique_pii)

freq
8


In [23]:
#Create a new relationship to connect clients that share identifiers and add the number of shared identifiers as a property on that relationship
shared_identifiers = """MATCH (c1:Client)-[:HAS_EMAIL|:HAS_PHONE|:HAS_SSN] ->(n)<- [:HAS_EMAIL|:HAS_PHONE|:HAS_SSN]-(c2:Client)
WHERE id(c1) < id(c2)
WITH c1, c2, count(*) as cnt
MERGE (c1) - [:SHARED_IDENTIFIERS {count: cnt}] -> (c2);"""

graph.run(shared_identifiers)

(No data)

In [24]:
shared_identifiers1 = """
MATCH p = (:Client) - [s:SHARED_IDENTIFIERS] -> (:Client) WHERE s.count >= 1 RETURN p 
"""

graph.run(shared_identifiers1)

p
(Kayden Payne)-[:SHARED_IDENTIFIERS {count: 1}]->(Tyler Turner)
(Tyler Leon)-[:SHARED_IDENTIFIERS {count: 2}]->(Aiden Burns)
(Lillian Atkins)-[:SHARED_IDENTIFIERS {count: 1}]->(Levi Moss)


In [25]:
graph1 = graph.run(shared_identifiers1).to_subgraph()

widget = ipycytoscape.CytoscapeWidget()
widget.graph.add_graph_from_neo4j(graph1)

style = [node_centered, edge_directed_named]
labels = list(graph1.labels())
print('Node labels:', labels)

colors = random_color_palette(len(labels))

for label, color in zip(labels, colors):
    style.append({'selector': 'node[label = "' + label + '"]', 'style': {'background-color': color}})
    
widget.set_layout(name='cola', padding=0, nodeSpacing=65, nodeDimensionsIncludeLabels=True, unconstrIter=5000)
widget.set_tooltip_source('tooltip')
widget

Node labels: ['Mule', 'Client']


CytoscapeWidget(cytoscape_layout={'name': 'cola', 'padding': 0, 'nodeSpacing': 65, 'nodeDimensionsIncludeLabel…

In [26]:
#Create project graph called 'wcc' to refer to gds library
gds = """
CALL gds.graph.project('wcc',
    {
        Client: {
            label: 'Client'
        }
    },
    {
        SHARED_IDENTIFIERS:{
            type: 'SHARED_IDENTIFIERS',
            orientation: 'UNDIRECTED',
            properties: {
                count: {
                    property: 'count'
                }
            }
        }
    }
) YIELD graphName,nodeCount,relationshipCount,projectMillis;"""

graph.run(gds)

graphName,nodeCount,relationshipCount,projectMillis
wcc,2129,8,876


In [27]:
#run Weakly connected components to find clusters of clients sharing PII.
wcc = """
CALL gds.wcc.stream('wcc',
    {
        nodeLabels: ['Client'],
        relationshipTypes: ['SHARED_IDENTIFIERS'],
        consecutiveIds: true
    }
)
YIELD nodeId, componentId
RETURN gds.util.asNode(nodeId).id AS clientId, componentId
ORDER BY componentId LIMIT 20"""

graph.run(wcc).to_data_frame()

,clientId,componentId
0,4403346685586374,0
1,4889592132732003,1
2,4876281590420864,2
3,4111314778042310,3
4,4756225504544454,4
5,4221203098728012,5
6,4948980857801641,6
7,4713875011658379,7
8,4905261316152319,8
9,4530290273653129,9


In [28]:
#write results back to database and label the node that has more than one cluster size as for potential first party fraud group
firstgroup = """
CALL gds.wcc.stream('wcc',
    {
        nodeLabels: ['Client'],
        relationshipTypes: ['SHARED_IDENTIFIERS'],
        consecutiveIds: true
    }
)
YIELD componentId, nodeId
WITH componentId AS cluster, gds.util.asNode(nodeId) AS client
WITH cluster, collect(client.id) AS clients
WITH cluster, clients, size(clients) AS clusterSize WHERE clusterSize > 1
UNWIND clients AS client
MATCH (c:Client) WHERE c.id = client
SET c.firstPartyFraudGroup=cluster;"""

graph.run(firstgroup)

(No data)

In [29]:
#project client nodes (one type) and three identifiers nodes (that are considered as second type) into 
#similarity memory graph to prepare for calculation using node similarity algorithm
graph_projection = """
MATCH(c:Client) WHERE c.firstPartyFraudGroup is not NULL
WITH collect(c) as clients
MATCH(n) WHERE n:Email OR n:Phone OR n:SSN
WITH clients, collect(n) as identifiers
WITH clients + identifiers as nodes

MATCH(c:Client) -[:HAS_EMAIL|:HAS_PHONE|:HAS_SSN]->(id)
WHERE c.firstPartyFraudGroup is not NULL
WITH nodes, collect({source: c, target: id}) as relationships

CALL gds.graph.project.cypher('similarity',
    "UNWIND $nodes as n RETURN id(n) AS id,labels(n) AS labels",
    "UNWIND $relationships as r RETURN id(r['source']) AS source, id(r['target']) AS target, 'HAS_IDENTIFIER' as type",
    { parameters: {nodes: nodes, relationships: relationships}}
)
YIELD graphName, nodeCount, relationshipCount, projectMillis
RETURN graphName, nodeCount, relationshipCount, projectMillis"""

graph.run(graph_projection)

graphName,nodeCount,relationshipCount,projectMillis
similarity,6390,24,2413


In [30]:
#mutate the similarity score calculated using jaccard metric into memory graph
mutate = """
CALL gds.nodeSimilarity.mutate('similarity',
    {
        topK:15,
        mutateProperty: 'jaccardScore',
        mutateRelationshipType:'SIMILAR_TO'
    }
);"""

graph.run(mutate)

preProcessingMillis,computeMillis,mutateMillis,postProcessingMillis,nodesCompared,relationshipsWritten,similarityDistribution,configuration
0,194,93,-1,8,8,"{p1: 0.19999980926513672, max: 0.5000028610229492, p5: 0.19999980926513672, p90: 0.5000028610229492, p50: 0.19999980926513672, p95: 0.5000028610229492, p10: 0.19999980926513672, p75: 0.19999980926513672, p99: 0.5000028610229492, p25: 0.19999980926513672, p100: 0.5000028610229492, min: 0.19999980926513672, mean: 0.27500033378601074, stdDev: 0.12990471906446924}","{topK: 15, similarityMetric: 'JACCARD', bottomK: 10, bottomN: 0, relationshipWeightProperty: null, mutateRelationshipType: 'SIMILAR_TO', topN: 0, concurrency: 4, degreeCutoff: 1, similarityCutoff: 1e-42, nodeLabels: ['*'], sudo: false, relationshipTypes: ['*'], mutateProperty: 'jaccardScore', username: null}"


In [31]:
mutate2 = """CALL gds.graph.writeRelationship('similarity', 'SIMILAR_TO', 'jaccardScore');"""

graph.run(mutate2)

writeMillis,graphName,relationshipType,relationshipProperty,relationshipsWritten,propertiesWritten
27,similarity,SIMILAR_TO,jaccardScore,8,8


In [32]:
#write the similarity score calculated in memory graph back to the real database
write_score = """
CALL gds.degree.write('similarity',
    {
        nodeLabels: ['Client'],
        relationshipTypes: ['SIMILAR_TO'],
        relationshipWeightProperty: 'jaccardScore',
        writeProperty: 'firstPartyFraudScore'
    }
);"""

graph.run(write_score)

nodePropertiesWritten,centralityDistribution,writeMillis,postProcessingMillis,preProcessingMillis,computeMillis,configuration
8,"{p99: 0.5000028610229492, min: 0.19999980926513672, max: 0.5000028610229492, mean: 0.27500033378601074, p90: 0.5000028610229492, p50: 0.19999980926513672, p999: 0.5000028610229492, p95: 0.5000028610229492, p75: 0.19999980926513672}",71,700,1,6,"{orientation: 'NATURAL', writeConcurrency: 4, writeProperty: 'firstPartyFraudScore', relationshipWeightProperty: 'jaccardScore', nodeLabels: ['Client'], sudo: false, relationshipTypes: ['SIMILAR_TO'], username: null, concurrency: 4}"


In [33]:
#label client having higher score than the threshold as a fraudster
firstfraudster = """
MATCH(c:Client)
WHERE c.firstPartyFraudScore IS NOT NULL
WITH percentileCont(c.firstPartyFraudScore, 0.80) AS firstPartyFraudThreshold

MATCH(c:Client)
WHERE c.firstPartyFraudScore > firstPartyFraudThreshold
SET c:FirstPartyFraudster;"""

graph.run(firstfraudster)

(No data)

In [34]:
#visualize the first party fraud groups
firstgroup_visual = """MATCH (c:Client)
WITH c.firstPartyFraudGroup AS fpGroupID, collect(c.id) AS fGroup
WITH *, size(fGroup) AS groupSize WHERE groupSize >= 1
WITH collect(fpGroupID) AS fraudRings
MATCH p=(c:Client)-[:HAS_SSN|HAS_EMAIL|HAS_PHONE]->()
WHERE c.firstPartyFraudGroup IN fraudRings
RETURN p"""

graph2 = graph.run(firstgroup_visual).to_subgraph()

widget2 = ipycytoscape.CytoscapeWidget()
widget2.graph.add_graph_from_neo4j(graph2)

labels2 = list(graph2.labels())
print('Node labels:', labels2)

colors2 = random_color_palette(len(labels2))

for label, color in zip(labels2, colors2):
    style.append({'selector': 'node[label = "' + label + '"]', 'style': {'background-color': color}})
    
widget2.set_layout(name='cola', padding=0, nodeSpacing=65, nodeDimensionsIncludeLabels=True, unconstrIter=5000)
widget2.set_tooltip_source('tooltip')
widget2

Node labels: ['SSN', 'Phone', 'Email', 'FirstPartyFraudster', 'Mule', 'Client']


CytoscapeWidget(cytoscape_layout={'name': 'cola', 'padding': 0, 'nodeSpacing': 65, 'nodeDimensionsIncludeLabel…

In [35]:
#Set labels between clients who had a hand in transaction with mules to be second party fraud suspect
secondfraudsuspect = """
MATCH (c1:Mule)<-[t:TRANSFER]-(c2:Client)
WHERE NOT c2:Mule
WITH c1, c2, sum(t.amount) AS totalAmount
SET c2:SecondPartyFraudSuspect
CREATE (c1)<-[:TRANSFER_TO {amount:totalAmount}]-(c2);"""

graph.run(secondfraudsuspect)

(No data)

In [36]:
delete_transfer = """
MATCH (p:Mule)<-[r:TRANSFER]-(c)
WHERE c:SecondPartyFraudSuspect
DELETE r"""

graph.run(delete_transfer)

(No data)

In [37]:
#create another projection graph with TRANSFER_TO relationships
transfer_to = """
CALL gds.graph.project('SecondPartyFraudNetwork',
    'Client',
    'TRANSFER_TO',
    {relationshipProperties:'amount'}
);"""

graph.run(transfer_to)

nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,projectMillis
"{Client: {label: 'Client', properties: {}}}","{TRANSFER_TO: {orientation: 'NATURAL', aggregation: 'DEFAULT', type: 'TRANSFER_TO', properties: {amount: {defaultValue: null, property: 'amount', aggregation: 'DEFAULT'}}}}",SecondPartyFraudNetwork,2129,239,293


In [38]:
#tag second fraud clusterid to groups that may have more than one client size using wcc
wcc2 = """
CALL gds.wcc.stream('SecondPartyFraudNetwork')
YIELD nodeId, componentId
WITH gds.util.asNode(nodeId) AS client, componentId AS clusterId
WITH clusterId, collect(client.id) AS cluster
WITH clusterId, size(cluster) AS clusterSize, cluster
WHERE clusterSize > 1
UNWIND cluster AS client
MATCH(c:Client {id:client})
SET c.secondPartyFraudGroup=clusterId;"""

graph.run(wcc2)

(No data)

In [39]:
#using pagerank algorithm to find second party fraud who has higher score - likely chance to commit 
#weighted by the amount of money transfered to fraud mules
pagerank = """
CALL gds.pageRank.stream('SecondPartyFraudNetwork',
    {relationshipWeightProperty:'amount'}
)YIELD nodeId, score
WITH gds.util.asNode(nodeId) AS client, score AS pageRankScore

WHERE client.secondPartyFraudGroup IS NOT NULL
        AND pageRankScore > 0 AND NOT client:Mule

MATCH(c:Client {id:client.id})
SET c:SecondPartyFraud
SET c.secondPartyFraudScore = pageRankScore;"""

graph.run(pagerank)

(No data)

In [43]:
#visualize the second party fraud groups
secondgroup_visual = """MATCH (p:Mule)<-[r:TRANSFER_TO]-(c)
WHERE c:SecondPartyFraud
RETURN p,c,r
LIMIT 40
"""

graph3 = graph.run(secondgroup_visual).to_subgraph()

widget3 = ipycytoscape.CytoscapeWidget()
widget3.graph.add_graph_from_neo4j(graph3)

labels3 = list(graph2.labels())
print('Node labels:', labels3)

colors3 = random_color_palette(len(labels3))

for label, color in zip(labels3, colors3):
    style.append({'selector': 'node[label = "' + label + '"]', 'style': {'background-color': color}})
    
widget3.set_layout(name='cola', padding=0, nodeSpacing=65, nodeDimensionsIncludeLabels=True, unconstrIter=5000)
widget3.set_tooltip_source('tooltip')
widget3

Node labels: ['SSN', 'Phone', 'Email', 'FirstPartyFraudster', 'Mule', 'Client']


CytoscapeWidget(cytoscape_layout={'name': 'cola', 'padding': 0, 'nodeSpacing': 65, 'nodeDimensionsIncludeLabel…